In [7]:
import os
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append("../src")

In [8]:
preselection_configs = [file for file in os.listdir("/afs/desy.de/user/s/spatarod/QUBO-V2/configuration/preselection/") 
                        if "c_" in file]
qubo_configs = [file for file in os.listdir("/afs/desy.de/user/s/spatarod/QUBO-V2/configuration/qubo/") 
                if ".ipynb" not in file]

In [9]:
qubo_configs

['bit_flip.yaml',
 'VQE_IdealQasmSim_10q_TwoLocal_NFT.yaml',
 'VQE_IdealQasmSim_12q_TwoLocal_NFT.yaml',
 'VQE_IdealQasmSim_16q_TwoLocal_NFT.yaml',
 'VQE_IdealQasmSim_3q_TwoLocal_NFT.yaml',
 'VQE_IdealQasmSim_5q_TwoLocal_NFT.yaml',
 'VQE_IdealQasmSim_7q_TwoLocal_COBYLA.yaml',
 'VQE_IdealQasmSim_7q_TwoLocal_NFT_circular.yaml',
 'VQE_IdealQasmSim_7q_TwoLocal_NFT_circular_deep.yaml',
 'VQE_IdealQasmSim_7q_TwoLocal_NFT_full.yaml',
 'VQE_IdealQasmSim_7q_TwoLocal_NFT_full_deep.yaml',
 'VQE_IdealQasmSim_7q_TwoLocal_NFT_linear.yaml',
 'VQE_IdealQasmSim_7q_TwoLocal_NFT_linear_deep.yaml',
 'VQE_IdealQasmSim_7q_TwoLocal_SPSA.yaml',
 'bit_flip_alt_version_1.yaml',
 'bit_flip_alt_version_2.yaml',
 'bit_flip_alt_version_3.yaml',
 'bit_flip_alt_version_4.yaml',
 'bit_flip_alt_version_5.yaml',
 'eigensolver_10q.yaml',
 'eigensolver_12q.yaml',
 'eigensolver_16q.yaml',
 'eigensolver_1q.yaml',
 'eigensolver_3q.yaml',
 'eigensolver_5q.yaml',
 'eigensolver_7q.yaml',
 'eigensolver_7q_alt_version_1.yaml',
 'e

In [36]:
prefix = "/nfs/dust/luxe/user/spatarod/towards_paper/e-laser/phase-0/gpc/"
xi = 3.0
preselection = "-c_4"
qubo_config = "eigensolver_7q" 
matched_definition = 3

def get_process_time(file):
    data = np.load(file, allow_pickle=True)
    print(len(list(data[()]["truth solution vector"])))
    return data[()]["time tracking complete"]["complete run"], list(data[()]["time tracking subQUBOs"].values())
    

def get_efficiency(reco, gen, num_hits_from_same_particle):
    xplet_map = {}
    
    reco = np.load(reco, allow_pickle=True)    
    gen = np.load(gen, allow_pickle=True)
    
    reco_temp = []                             
    reco_numbers = set()                      
    
    for number, xplet in enumerate(reco):  
        for hit_id in xplet.hit_ids.values():         
            if hit_id in xplet_map.keys():            
                values = xplet_map[hit_id]            
                values.append(number)                 
                xplet_map.update({hit_id: values})
            else:
                xplet_map.update({hit_id: [number]})
     
    for reco_xplet in reco:
        unique = True
        for hit_id in reco_xplet.hit_ids.values():
            if len(list(xplet_map[hit_id])) > 1:
                unique = False

        if unique:
            reco_temp.append(reco_xplet)
    reco_temp = reco
    fake = 0
    matched = set()
    
    for xplet in reco_temp:
        matched_xplet = False
        for particle in set(xplet.particle_ids.values()):
            if list(xplet.particle_ids.values()).count(particle) >= num_hits_from_same_particle:
                matched_xplet = True
        if matched_xplet:
            matched.add(particle)
        else:
            fake += 1
    return len(matched) / len(gen), fake / len(reco)

In [37]:
efficiency = []
fake_rate = []

for folder in os.listdir(f"{prefix}{xi}/smeared/"):
    if preselection in folder:
        for qubo in os.listdir(f"{prefix}{xi}/smeared/{folder}"):
            if "_".join(qubo.split("_")[1:]) == qubo_config:
                bx = folder.split("_")[2]
                print(f"{prefix}{xi}/smeared/{folder}/{qubo}")
                eff, fake = get_efficiency(f"{prefix}{xi}/smeared/{folder}/{qubo}/reco_xplet_list_v2.npy",
                                           f"{prefix}{xi}/e0gpc_{xi}_{bx}_sl_gen_xplet_list.npy",
                                           matched_definition)
                efficiency.append(eff)
                fake_rate.append(fake)

print(f"xi = {xi}")
print(f"Efficiency {np.around(np.mean(efficiency), 3)}", "+/-", np.around(np.std(efficiency), 3))
print(f"Fake rate: {np.around(np.mean(fake_rate), 3)}", "+/-", np.around(np.std(fake_rate), 3))

/nfs/dust/luxe/user/spatarod/towards_paper/e-laser/phase-0/gpc/3.0/smeared/e0gpc_3.0_0008_sl-c_4/386678389_eigensolver_7q
/nfs/dust/luxe/user/spatarod/towards_paper/e-laser/phase-0/gpc/3.0/smeared/e0gpc_3.0_0006_sl-c_4/442363015_eigensolver_7q
/nfs/dust/luxe/user/spatarod/towards_paper/e-laser/phase-0/gpc/3.0/smeared/e0gpc_3.0_0003_sl-c_4/430362797_eigensolver_7q
/nfs/dust/luxe/user/spatarod/towards_paper/e-laser/phase-0/gpc/3.0/smeared/e0gpc_3.0_0009_sl-c_4/482239023_eigensolver_7q
/nfs/dust/luxe/user/spatarod/towards_paper/e-laser/phase-0/gpc/3.0/smeared/e0gpc_3.0_0000_sl-c_4/510758538_eigensolver_7q
/nfs/dust/luxe/user/spatarod/towards_paper/e-laser/phase-0/gpc/3.0/smeared/e0gpc_3.0_0001_sl-c_4/596569879_eigensolver_7q
/nfs/dust/luxe/user/spatarod/towards_paper/e-laser/phase-0/gpc/3.0/smeared/e0gpc_3.0_0002_sl-c_4/780648100_eigensolver_7q
/nfs/dust/luxe/user/spatarod/towards_paper/e-laser/phase-0/gpc/3.0/smeared/e0gpc_3.0_0004_sl-c_4/801872441_eigensolver_7q
/nfs/dust/luxe/user/spat

In [16]:
efficiency = []
fake_rate = []

for folder in os.listdir(f"{prefix}{xi}/smeared/"):
    if preselection in folder:
        for qubo in os.listdir(f"{prefix}{xi}/smeared/{folder}"):
            if "_".join(qubo.split("_")[1:]) == qubo_config:
                bx = folder.split("_")[2]
                try:
                    print(f"{prefix}{xi}/smeared/{folder}/{qubo}")
                    eff, fake = get_efficiency(f"{prefix}{xi}/smeared/{folder}/{qubo}/reco_xplet_list_22.npy",
                                               f"{prefix}{xi}/e0gpc_{xi}_{bx}_sl_gen_xplet_list.npy",
                                               matched_definition)
                    efficiency.append(eff)
                    fake_rate.append(fake)
                except:
                    print(f"{prefix}{xi}/smeared/{folder}/{qubo}")
                    pass
print(f"xi = {xi}")
print(f"Efficiency {np.around(np.mean(efficiency), 3)}", "+/-", np.around(np.std(efficiency), 3))
print(f"Fake rate: {np.around(np.mean(fake_rate), 3)}", "+/-", np.around(np.std(fake_rate), 3))

/nfs/dust/luxe/user/spatarod/towards_paper/e-laser/phase-0/gpc/5.0/smeared/e0gpc_5.0_0001_sl-c_4/545090974_VQE_IdealQasmSim_7q_TwoLocal_NFT
/nfs/dust/luxe/user/spatarod/towards_paper/e-laser/phase-0/gpc/5.0/smeared/e0gpc_5.0_0001_sl-c_4/545090974_VQE_IdealQasmSim_7q_TwoLocal_NFT
/nfs/dust/luxe/user/spatarod/towards_paper/e-laser/phase-0/gpc/5.0/smeared/e0gpc_5.0_0004_sl-c_4/981100586_VQE_IdealQasmSim_7q_TwoLocal_NFT
/nfs/dust/luxe/user/spatarod/towards_paper/e-laser/phase-0/gpc/5.0/smeared/e0gpc_5.0_0004_sl-c_4/981100586_VQE_IdealQasmSim_7q_TwoLocal_NFT
/nfs/dust/luxe/user/spatarod/towards_paper/e-laser/phase-0/gpc/5.0/smeared/e0gpc_5.0_0000_sl-c_4/720797590_VQE_IdealQasmSim_7q_TwoLocal_NFT
/nfs/dust/luxe/user/spatarod/towards_paper/e-laser/phase-0/gpc/5.0/smeared/e0gpc_5.0_0000_sl-c_4/720797590_VQE_IdealQasmSim_7q_TwoLocal_NFT
/nfs/dust/luxe/user/spatarod/towards_paper/e-laser/phase-0/gpc/5.0/smeared/e0gpc_5.0_0002_sl-c_4/620571882_VQE_IdealQasmSim_7q_TwoLocal_NFT
/nfs/dust/luxe/user/

/usr/local/lib64/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib64/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib64/python3.6/site-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib64/python3.6/site-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib64/python3.6/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


size = 1:
Efficiency 0.931 +/- 0.002
Fake rate: 0.296 +/- 0.009

size = 3
Efficiency 0.929 +/- 0.003
Fake rate: 0.03 +/- 0.002

size = 5:
Efficiency 0.929 +/- 0.003
Fake rate: 0.03 +/- 0.002

size = 7:
Efficiency 0.929 +/- 0.004
Fake rate: 0.03 +/- 0.002

size = 10:
Efficiency 0.929 +/- 0.004
Fake rate: 0.03 +/- 0.002

size = 12:
Efficiency 0.929 +/- 0.003
Fake rate: 0.03 +/- 0.002

size = 16:
Efficiency 0.929 +/- 0.003
Fake rate: 0.03 +/- 0.002

In [ ]:
complete_time = []
sub_QUBO_time = []


for folder in os.listdir(f"{prefix}{xi}/smeared/"):
    if preselection in folder:
        for qubo in os.listdir(f"{prefix}{xi}/smeared/{folder}"):
            if "_".join(qubo.split("_")[1:]) == qubo_config:
                bx = folder.split("_")[2]
                try:
                    print(f"{prefix}{xi}/smeared/{folder}/{qubo}")
                    complete, sub_QUBO = get_process_time(f"{prefix}{xi}/smeared/{folder}/{qubo}/qubo_log.npy")
                    
                    complete_time.append(complete)
                    for sub in sub_QUBO:
                        sub_QUBO_time.append(sub)
                except:
                    pass
                #     print(f"{prefix}{xi}/smeared/{folder}/{qubo}")
                #     pass
print(f"xi = {xi}")
print(f"Complete {np.around(np.mean(complete_time), 4)}", "+/-", np.around(np.std(complete_time), 9))
print(f"sub-QUBO: {np.around(np.mean(sub_QUBO_time), 4)}", "+/-", np.around(np.std(sub_QUBO_time), 3))
print(complete_time)

In [34]:
prefix = "/nfs/dust/luxe/user/spatarod/towards_paper/e-laser/phase-0/gpc/"
xi = 3.0


data = np.load(f"{prefix}{xi}/e0gpc_{xi}_00{90}_sl_gen_xplet_list.npy", allow_pickle=True)

In [37]:
print(len(data))

125
